In [ ]:
%load_ext autoreload
%autoreload 2

from src.data.mocap import MotionCaptureDataset
from src.models.mocap import SphericalVAE, VariationalAutoencoder
from sklearn.decomposition import PCA
from src.utils import plot_3d
import torch

In [ ]:
dataset = MotionCaptureDataset("07") ## Salsa
X = dataset.X

In [ ]:
import re
pattern = re.compile("_(\d+):")
classes = [int(pattern.search(s).group(1)) for s in dataset.labels]

In [ ]:
x, y, z = PCA(n_components=3).fit_transform(X).T
fig = plot_3d(x, y, z, classes = classes)
fig.show()


In [ ]:
feature_dim = X.shape[-1]

svae = SphericalVAE(feature_dim=feature_dim, latent_dim=3)
svae.to(torch.double)
svae_state_dict = torch.load('../models/mocap/svae.pt')
svae.load_state_dict(svae_state_dict)

vae = VariationalAutoencoder(feature_dim, 3)
vae.to(torch.double)
vae_state_dict = torch.load('../models/mocap/vae.pt')
vae.load_state_dict(vae_state_dict)

In [ ]:
output = vae(X)
plot_3d( *output["z"].T.detach().numpy(), classes)

In [ ]:
output = svae(X)
plot_3d( *output["z"].T.detach().numpy(), classes)